<a href="https://colab.research.google.com/github/koalalovepabro/Stock-Portfolio-Optimization/blob/main/Build_A_Killer_Stock_Portfolio(KOSPI_new).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최적화된 주식 포트폴리오 만들기

## 💘 KOSPI 상장종목에서 KRX 관리종목 제거

In [2]:
!pip install -q install finance-datareader

In [3]:
import pandas as pd
import numpy as np
import requests

In [4]:
# Finance Datareader 불러오고, 버전 확인하기
import FinanceDataReader as fdr
fdr.__version__

'0.9.31'

In [5]:
# KRX : KRX 종목 전체 (KOSPI + KOSDAQ + KONEX)
# KRX100	: KRX 100
# KOSPI : KOSPI 종목
# KS100	: KOSPI 100
# KS200	: KOSPI 200
# KOSDAQ : KOSDAQ 종목
# KONEX : KONEX 종목

# KS11 : KOSPI 지수
# KQ11 : KOSDAQ 지수
# KS50 :	KOSPI 50 지수

# NASDAQ : 나스닥 종목
# NYSE : 뉴욕증권거래소 종목
# SP500 : S&P500 종목

💥 **KRX 관리종목은 사전에 삭제되어야 함 !
( 거래가 일시중단된 경우가 있음 )**



In [6]:
# KRX stock delisting symbol list and names 관리종목 리스트
krx_adm = fdr.StockListing('KRX-ADMINISTRATIVE') # 관리종목
krx_adm

,Symbol,Name,DesignationDate,Reason
0,002420,세기상사,2021-03-18,매출액 미달(50억원 미만)
1,002630,오리엔트바이오,2021-06-29,감사범위제한으로인한 감사의견한정
2,003280,흥아해운,2020-08-18,"기타,반기검토의견 의견거절"
3,003620,쌍용차,2021-04-16,"반기검토의견 의견거절,회생절차개시신청,기타"
4,007630,폴루스바이오팜,2020-05-18,"기타,감사범위제한으로인한 감사의견한정,반기검토의견 의견거절"
...,...,...,...,...
111,299910,베스파,2021-03-24,최근3사업연도중 2사업연도 자기자본 50%초과 법인세비용차감전계속사업손실 발생
112,311270,키움제5호스팩,2021-06-21,SPAC 상장예비심사청구서 미제출 등
113,900290,GRT,2020-10-28,"상장폐지사유 발생,반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정"
114,900100,애머릿지,2020-04-24,상장적격성 실질심사 대상 결정


In [7]:
print(krx_adm.loc[krx_adm['Symbol'] == '010580'])

   Symbol Name DesignationDate                   Reason
5  010580   지코      2020-06-24  회생절차개시신청,반기검토의견 의견거절,기타


In [8]:
# KRX 관리종목 전체 가져오기
df_kospi = fdr.StockListing('KOSPI')
df_kospi

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,152100,KOSPI,ARIRANG 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6457,215620,KOSPI,흥국 S&P코리아로우볼,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6459,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시
6460,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6461,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [9]:
krx_adm['Symbol'].values

array(['002420', '002630', '003280', '003620', '007630', '010580',
       '011300', '012600', '013000', '015540', '096760', '234080',
       '291660', '296710', '006580', '009730', '014100', '016670',
       '024830', '032790', '033110', '033340', '033430', '033600',
       '033790', '035290', '038060', '038160', '038340', '039230',
       '039310', '039670', '043200', '043590', '044490', '045890',
       '046070', '050320', '050540', '052190', '052770', '053110',
       '053660', '054220', '056000', '056730', '058220', '058420',
       '058450', '058530', '060300', '060900', '064090', '064510',
       '064520', '065150', '065420', '065560', '065620', '066110',
       '069110', '069330', '069540', '078590', '079190', '080440',
       '083470', '083660', '086250', '087730', '095270', '099410',
       '099520', '101400', '101680', '101730', '102940', '103230',
       '106080', '106520', '111820', '114120', '114190', '114570',
       '115180', '115530', '115610', '121890', '122690', '1268

In [10]:
# KOSPI 목록에 포함된 KRX 관리종목 찾기
for i in krx_adm['Symbol'].values:
  for k in df_kospi['Symbol'].values:
    if i == k:
      print(i)

002420
002630
003280
003620
007630
010580
011300
012600
013000
015540
096760
234080
291660
296710


In [11]:
danger = []
# KOSPI 목록에 포함된 KRX 관리종목을 리스트로 저장
for i in krx_adm['Symbol'].values:
  for k in df_kospi['Symbol'].values:
    if i == k:
      danger.append(i)
danger

['002420',
 '002630',
 '003280',
 '003620',
 '007630',
 '010580',
 '011300',
 '012600',
 '013000',
 '015540',
 '096760',
 '234080',
 '291660',
 '296710']

In [12]:
# KOSPI 목록에 포함된 KRX 관리종목의 정보 확인
for d in danger:
  print(krx_adm.loc[krx_adm['Symbol'] == d])
  print('-'*100)

   Symbol  Name DesignationDate           Reason
0  002420  세기상사      2021-03-18  매출액 미달(50억원 미만)
----------------------------------------------------------------------------------------------------
   Symbol     Name DesignationDate             Reason
1  002630  오리엔트바이오      2021-06-29  감사범위제한으로인한 감사의견한정
----------------------------------------------------------------------------------------------------
   Symbol  Name DesignationDate          Reason
2  003280  흥아해운      2020-08-18  기타,반기검토의견 의견거절
----------------------------------------------------------------------------------------------------
   Symbol Name DesignationDate                   Reason
3  003620  쌍용차      2021-04-16  반기검토의견 의견거절,회생절차개시신청,기타
----------------------------------------------------------------------------------------------------
   Symbol     Name DesignationDate                            Reason
4  007630  폴루스바이오팜      2020-05-18  기타,감사범위제한으로인한 감사의견한정,반기검토의견 의견거절
--------------------------------------------

In [13]:
df_kospi

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,152100,KOSPI,ARIRANG 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6457,215620,KOSPI,흥국 S&P코리아로우볼,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6459,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시
6460,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6461,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [14]:
idx_nm = []
# KOSPI 목록에 포함된 KRX 관리종목의 인덱스 찾기
for d in danger:
  for k in df_kospi['Symbol'].values:
    if d == k:
      idx_nm.append(df_kospi[df_kospi['Symbol'] == d].index)
idx_nm      

[Int64Index([3498], dtype='int64'),
 Int64Index([4059], dtype='int64'),
 Int64Index([6462], dtype='int64'),
 Int64Index([3714], dtype='int64'),
 Int64Index([4809], dtype='int64'),
 Int64Index([4489], dtype='int64'),
 Int64Index([3490], dtype='int64'),
 Int64Index([3530], dtype='int64'),
 Int64Index([3510], dtype='int64'),
 Int64Index([3722], dtype='int64'),
 Int64Index([161], dtype='int64'),
 Int64Index([156], dtype='int64'),
 Int64Index([1358], dtype='int64'),
 Int64Index([1360], dtype='int64')]

In [15]:
len(idx_nm)

14

In [16]:
idx_nm[0][0]

3498

In [17]:
idx_nm[1][0]

4059

In [18]:
idx_nm_list = []
for i in range (0,len(idx_nm)):
  idx_nm_list.append(idx_nm[i][0])
idx_nm_list

[3498,
 4059,
 6462,
 3714,
 4809,
 4489,
 3490,
 3530,
 3510,
 3722,
 161,
 156,
 1358,
 1360]

In [19]:
len(idx_nm_list)

14

In [20]:
# KOSPI 목록에 포함된 KRX 관리종목 삭제
df_kospi_new = df_kospi.drop(idx_nm_list)

In [21]:
df_kospi_new

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,152100,KOSPI,ARIRANG 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6454,069260,KOSPI,휴켐스,기타 화학제품 제조업,"화합물,화학제품 제조",2002-10-07,12월,신진용,http://www.huchems.com,서울특별시
6457,215620,KOSPI,흥국 S&P코리아로우볼,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6459,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시
6460,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [22]:
# 제대로 삭제 됐는지 확인
# KOSPI 목록에 포함된 KRX 관리종목 찾기
for i in krx_adm['Symbol'].values:
  for k in df_kospi_new['Symbol'].values:
    if i == k:
      print(i)

In [23]:
# 데이터 파악
# 데이터의 결측치 존재 여부, 데이터의 타입, 컬럼의 수, 데이터프레임의 메모리 사용량 등
df_kospi_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4787 entries, 1 to 6461
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          4787 non-null   object        
 1   Market          4787 non-null   object        
 2   Name            4787 non-null   object        
 3   Sector          798 non-null    object        
 4   Industry        785 non-null    object        
 5   ListingDate     798 non-null    datetime64[ns]
 6   SettleMonth     798 non-null    object        
 7   Representative  798 non-null    object        
 8   HomePage        769 non-null    object        
 9   Region          798 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 411.4+ KB


In [24]:
# 데이터의 결측치 확인
df_kospi_new.isnull().sum()

Symbol               0
Market               0
Name                 0
Sector            3989
Industry          4002
ListingDate       3989
SettleMonth       3989
Representative    3989
HomePage          4018
Region            3989
dtype: int64

In [25]:
# 결측치 제거
df_kospi_new = df_kospi_new.dropna()
df_kospi_new.isnull().sum()

Symbol            0
Market            0
Name              0
Sector            0
Industry          0
ListingDate       0
SettleMonth       0
Representative    0
HomePage          0
Region            0
dtype: int64

In [26]:
# 데이터프레임의 통계정보
df_kospi_new.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
count,760,760,760,760,760,760,760,760,760,760
unique,760,1,760,126,721,607,7,743,748,19
top,227840,KOSPI,CJ대한통운,기타 금융업,지주회사,1976-06-30 00:00:00,12월,윤석빈,http://www.sempio.com,서울특별시
freq,1,760,1,55,17,8,738,2,2,387
first,NaN,NaN,NaN,NaN,NaN,1956-03-03 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2021-07-16 00:00:00,NaN,NaN,NaN,NaN


In [27]:
df_kospi_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 1 to 6459
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          760 non-null    object        
 1   Market          760 non-null    object        
 2   Name            760 non-null    object        
 3   Sector          760 non-null    object        
 4   Industry        760 non-null    object        
 5   ListingDate     760 non-null    datetime64[ns]
 6   SettleMonth     760 non-null    object        
 7   Representative  760 non-null    object        
 8   HomePage        760 non-null    object        
 9   Region          760 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 65.3+ KB


In [28]:
# 종목코드 가져오기
assets = df_kospi_new['Symbol']
assets

1       095570
2       006840
46      027410
47      282330
48      138930
         ...  
6438    005870
6447    079980
6449    005010
6454    069260
6459    000540
Name: Symbol, Length: 760, dtype: object

In [29]:
assets = np.array(assets)
assets

array(['095570', '006840', '027410', '282330', '138930', '001460',
       '001040', '079160', '000120', '011150', '097950', '000590',
       '012030', '016610', '005830', '000990', '139130', '001530',
       '000210', '001880', '004840', '155660', '069730', '017940',
       '365550', '383220', '007700', '114090', '006360', '001250',
       '007070', '012630', '039570', '089470', '294870', '011200',
       '082740', '003560', '175330', '001060', '105560', '002380',
       '344820', '009070', '009440', '119650', '092220', '016380',
       '001390', '033180', '001940', '025000', '092230', '000040',
       '030200', '033780', '030210', '058850', '058860', '093050',
       '003550', '034220', '051900', '032640', '011070', '066570',
       '037560', '051910', '079550', '006260', '010120', '000680',
       '001120', '108670', '383800', '023150', '035420', '181710',
       '005940', '338100', '034310', '008260', '004250', '010060',
       '005490', '010950', '034120', '101060', '005090', '0013

In [30]:
len(assets)

760

In [31]:
# 종목별 종가 가져오기
from datetime import datetime

# 주식 시작일은 2013년 1월 1일이고 종료일은 현재 날짜 (오늘)로 설정
#Get the stock starting date
start_date = '2013-01-01'
# today = datetime.today().strftime('%Y-%m-%d')
end_date = '2021-07-15'

# 각 주식의 일별 종가 데이터를 저장할 데이터 프레임을 생성
#Create a dataframe to store the adjusted close price of the stocks
df = pd.DataFrame()

# FinanceDataReader로 각 종목의 종가데이터 불러오기
for stock in assets:
  df[stock] = fdr.DataReader(stock, start_date, end_date)['Close']
df

,095570,006840,027410,282330,138930,001460,001040,079160,000120,011150,097950,000590,012030,016610,005830,000990,139130,001530,000210,001880,004840,155660,069730,017940,365550,383220,007700,114090,006360,001250,007070,012630,039570,089470,294870,011200,082740,003560,175330,001060,...,017800,307950,011210,267260,004020,267250,005380,001500,011760,227840,126560,001450,057050,093240,003010,111110,008770,002460,378850,006060,013520,010690,133820,010660,000850,016580,032560,004800,094280,298040,298050,298020,298000,093370,081660,005870,079980,005010,069260,000540
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-21,7100,90400,47810,NaN,14032,337500,256958,97511,171500,3900,391000,80000,627,4220,57200,15150,10000,60217,57380,21500,11981,3430,3820,67400,NaN,NaN,8250,27800,20850,4395,60100,41415,NaN,8300,NaN,32287,3064,2575,6590,23390,...,38814,NaN,95400,NaN,51200,NaN,146000,10150,27267,NaN,3530,28500,126500,2690,2935,NaN,110500,18550,NaN,5510,2406,5430,2180,2940,64200,22600,4104,76341,15000,NaN,NaN,NaN,NaN,3715,23000,10500,7850,16150,20900,3805
2015-08-24,6480,88800,45620,NaN,13167,334000,249886,94336,164500,3910,375000,80500,597,4105,54300,14050,9660,57780,55120,20550,11798,3400,3735,67100,NaN,NaN,8100,29000,21000,4089,59000,38292,NaN,7980,NaN,41954,3047,2500,6470,23688,...,39481,NaN,92700,NaN,50600,NaN,142500,9780,25068,NaN,3405,27800,121000,2871,2900,NaN,112000,18600,NaN,5920,2244,5220,1975,2765,62000,21750,4086,71607,14650,NaN,NaN,NaN,NaN,3700,21500,9570,7910,15650,20200,3650
2015-08-25,6220,92200,47166,NaN,13071,327500,253186,98418,167000,4310,372000,82500,614,4360,51700,15200,9620,60217,56295,21250,12434,3430,3895,66800,NaN,NaN,8400,29050,22750,4309,59800,41266,NaN,8530,NaN,45241,3043,2715,6550,25516,...,41348,NaN,99300,NaN,51400,NaN,147500,10050,25024,NaN,3405,27000,121000,3015,2960,NaN,115000,19500,NaN,6180,2410,5310,2135,2790,64000,21850,4318,73678,16100,NaN,NaN,NaN,NaN,3695,20600,9470,8000,16050,20350,3985
2015-08-26,7090,91700,50001,NaN,13311,339500,270631,105675,170500,4330,390000,82500,637,4460,52900,15950,10000,60669,59367,22150,12661,3480,4195,67000,NaN,NaN,8875,29150,23600,4444,62300,42381,NaN,8790,NaN,55422,3092,3130,6350,26791,...,44283,NaN,103500,NaN,52200,NaN,150000,10300,25771,NaN,3480,28150,121000,3052,3065,NaN,120000,19150,NaN,6300,2413,5480,2315,2835,64200,23850,4463,79004,17750,NaN,NaN,NaN,NaN,3860,20300,9300,8180,16450,21550,3905
2015-08-27,7570,91000,49743,NaN,13215,330000,267331,103861,167000,4300,393000,81000,644,4360,52700,15750,10050,60940,58373,21450,12888,3550,4130,66900,NaN,NaN,9050,30250,23550,4444,64200,42158,NaN,9140,NaN,52974,3096,3100,6390,26537,...,43749,NaN,103000,NaN,51800,NaN,147000,10150,25332,NaN,3660,28600,126000,2983,3070,NaN,125000,19450,NaN,6520,2541,5740,2400,2905,64200,22800,4585,77229,18100,NaN,NaN,NaN,NaN,3855,21000,10850,8260,17000,21350,3925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,5850,27650,6160,163000.0,7350,390500,98400,29400,181500,5040,442500,67800,1225,6720,53500,60600,8810,195000,77300,35450,7590,6680,6700,55400,7650.0,527000.0,35950,16350,42050,2815,36350,12900,13600.0,5910,32100.0,44650,9040,1825,7180,30150,...,49800,128000.0,91400,19250.0,51600,66600.0,226000,12900,19950,14950.0,4325,24700,86200,5190,6800,12900.0,92700,13400,5980.0,7380,3150,10050,2795,2755,37350,22150,9120,104500,21850,70100.0,513000.0,890000.0,381500.0,11450,53000,8040,10650,13800,22400,4340
2021-07-12,5910,28550,6260,167000.0,7520,395000,101000,29900,181000,4965,443000,68500,1255,6980,55000,64300,9030,195500,79200,35850,7610,6870,6630,56000,7630.0,535000.0,36050,16450,43600,2915,36450,13100,13750.0,6000,32750.0,43800,9280,1860,7300,30300,...,50500,128000.0,91300,2

In [32]:
# DataFrame을 csv 파일로 저장하기 ( 결측값 제거하지 않음 )
df.to_csv("kospi_new_code_close.csv", index=True)

In [33]:
# 칼럼명을 회사이름으로 변경
df.columns = df_kospi_new['Name'].values
df

,AJ네트웍스,AK홀딩스,BGF,BGF리테일,BNK금융지주,BYC,CJ,CJ CGV,CJ대한통운,CJ씨푸드,CJ제일제당,CS홀딩스,DB,DB금융투자,DB손해보험,DB하이텍,DGB금융지주,DI동일,DL,DL건설,DRB동일,DSR,DSR제강,E1,ESR켄달스퀘어리츠,F&F,F&F홀딩스,GKL,GS건설,GS글로벌,GS리테일,HDC,HDC아이콘트롤스,HDC현대EP,HDC현대산업개발,HMM,HSD엔진,IHQ,JB금융지주,JW중외제약,...,현대엘리베이,현대오토에버,현대위아,현대일렉트릭,현대제철,현대중공업지주,현대차,현대차증권,현대코퍼레이션,현대코퍼레이션홀딩스,현대퓨처넷,현대해상,현대홈쇼핑,형지엘리트,혜인,호전실업,호텔신라,화성산업,화승알앤에이,화승인더,화승코퍼레이션,화신,화인베스틸,화천기계,화천기공,환인제약,황금에스티,효성,효성ITX,효성중공업,효성첨단소재,효성티앤씨,효성화학,후성,휠라홀딩스,휴니드,휴비스,휴스틸,휴켐스,흥국화재
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-21,7100,90400,47810,NaN,14032,337500,256958,97511,171500,3900,391000,80000,627,4220,57200,15150,10000,60217,57380,21500,11981,3430,3820,67400,NaN,NaN,8250,27800,20850,4395,60100,41415,NaN,8300,NaN,32287,3064,2575,6590,23390,...,38814,NaN,95400,NaN,51200,NaN,146000,10150,27267,NaN,3530,28500,126500,2690,2935,NaN,110500,18550,NaN,5510,2406,5430,2180,2940,64200,22600,4104,76341,15000,NaN,NaN,NaN,NaN,3715,23000,10500,7850,16150,20900,3805
2015-08-24,6480,88800,45620,NaN,13167,334000,249886,94336,164500,3910,375000,80500,597,4105,54300,14050,9660,57780,55120,20550,11798,3400,3735,67100,NaN,NaN,8100,29000,21000,4089,59000,38292,NaN,7980,NaN,41954,3047,2500,6470,23688,...,39481,NaN,92700,NaN,50600,NaN,142500,9780,25068,NaN,3405,27800,121000,2871,2900,NaN,112000,18600,NaN,5920,2244,5220,1975,2765,62000,21750,4086,71607,14650,NaN,NaN,NaN,NaN,3700,21500,9570,7910,15650,20200,3650
2015-08-25,6220,92200,47166,NaN,13071,327500,253186,98418,167000,4310,372000,82500,614,4360,51700,15200,9620,60217,56295,21250,12434,3430,3895,66800,NaN,NaN,8400,29050,22750,4309,59800,41266,NaN,8530,NaN,45241,3043,2715,6550,25516,...,41348,NaN,99300,NaN,51400,NaN,147500,10050,25024,NaN,3405,27000,121000,3015,2960,NaN,115000,19500,NaN,6180,2410,5310,2135,2790,64000,21850,4318,73678,16100,NaN,NaN,NaN,NaN,3695,20600,9470,8000,16050,20350,3985
2015-08-26,7090,91700,50001,NaN,13311,339500,270631,105675,170500,4330,390000,82500,637,4460,52900,15950,10000,60669,59367,22150,12661,3480,4195,67000,NaN,NaN,8875,29150,23600,4444,62300,42381,NaN,8790,NaN,55422,3092,3130,6350,26791,...,44283,NaN,103500,NaN,52200,NaN,150000,10300,25771,NaN,3480,28150,121000,3052,3065,NaN,120000,19150,NaN,6300,2413,5480,2315,2835,64200,23850,4463,79004,17750,NaN,NaN,NaN,NaN,3860,20300,9300,8180,16450,21550,3905
2015-08-27,7570,91000,49743,NaN,13215,330000,267331,103861,167000,4300,393000,81000,644,4360,52700,15750,10050,60940,58373,21450,12888,3550,4130,66900,NaN,NaN,9050,30250,23550,4444,64200,42158,NaN,9140,NaN,52974,3096,3100,6390,26537,...,43749,NaN,103000,NaN,51800,NaN,147000,10150,25332,NaN,3660,28600,126000,2983,3070,NaN,125000,19450,NaN,6520,2541,5740,2400,2905,64200,22800,4585,77229,18100,NaN,NaN,NaN,NaN,3855,21000,10850,8260,17000,21350,3925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,5850,27650,6160,163000.0,7350,390500,98400,29400,181500,5040,442500,67800,1225,6720,53500,60600,8810,195000,77300,35450,7590,6680,6700,55400,7650.0,527000.0,35950,16350,42050,2815,36350,12900,13600.0,5910,32100.0,44650,9040,1825,7180,30150,...,49800,128000.0,91400,19250.0,51600,66600.0,226000,12900,19950,14950.0,4325,24700,86200,5190,6800,12900.0,92700,13400,5980.0,7380,3150,10050,2795,2755,37350,22150,9120,104500,21850,70100.0,513000.0,890000.0,381500.0,11450,53000,8040,10650,13800,22400,4340
2021-07-12,5910,28550,6260,167000.0,7520,395000,101000,29900,181000,4965,443000,68500,1255,6980,55000,64300,9030,195500,79200,35850,7610,6870,6630,56000,7630.0,535000.0,36050,16450,43600,2915,36450,13100,13750.0,6000,32750.0,43800,9280,1860,7300,30300,...,50500,128000.0,91300,20400.0,53100,68300.0,230000,13150,20000,15150.0,4370,25300,87000,5200,6950,13500.0,93300,13600,6000.

In [34]:
# 결측값 있는 열 삭제  ( 종목 1451  -> 672으로 줄어 듦 )
df2 = df.dropna(axis = 1)
df2

,AJ네트웍스,AK홀딩스,BGF,BNK금융지주,BYC,CJ,CJ CGV,CJ대한통운,CJ씨푸드,CJ제일제당,CS홀딩스,DB,DB금융투자,DB손해보험,DB하이텍,DGB금융지주,DI동일,DL,DL건설,DRB동일,DSR,DSR제강,E1,F&F홀딩스,GKL,GS건설,GS글로벌,GS리테일,HDC,HDC현대EP,HMM,HSD엔진,IHQ,JB금융지주,JW중외제약,KB금융,KCC,KCTC,KC그린홀딩스,KC코트렐,...,현대건설,현대그린푸드,현대글로비스,현대로템,현대리바트,현대모비스,현대미포조선,현대백화점,현대비앤지스틸,현대약품,현대엘리베이,현대위아,현대제철,현대차,현대차증권,현대코퍼레이션,현대퓨처넷,현대해상,현대홈쇼핑,형지엘리트,혜인,호텔신라,화성산업,화승인더,화승코퍼레이션,화신,화인베스틸,화천기계,화천기공,환인제약,황금에스티,효성,효성ITX,후성,휠라홀딩스,휴니드,휴비스,휴스틸,휴켐스,흥국화재
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-21,7100,90400,47810,14032,337500,256958,97511,171500,3900,391000,80000,627,4220,57200,15150,10000,60217,57380,21500,11981,3430,3820,67400,8250,27800,20850,4395,60100,41415,8300,32287,3064,2575,6590,23390,36200,317816,2880,6650,5567,...,30850,24600,175000,15950,55700,200500,23086,149000,9350,3093,38814,95400,51200,146000,10150,27267,3530,28500,126500,2690,2935,110500,18550,5510,2406,5430,2180,2940,64200,22600,4104,76341,15000,3715,23000,10500,7850,16150,20900,3805
2015-08-24,6480,88800,45620,13167,334000,249886,94336,164500,3910,375000,80500,597,4105,54300,14050,9660,57780,55120,20550,11798,3400,3735,67100,8100,29000,21000,4089,59000,38292,7980,41954,3047,2500,6470,23688,34900,314304,2815,6760,5274,...,30450,23850,163000,16300,53300,194500,22610,143500,9250,3079,39481,92700,50600,142500,9780,25068,3405,27800,121000,2871,2900,112000,18600,5920,2244,5220,1975,2765,62000,21750,4086,71607,14650,3700,21500,9570,7910,15650,20200,3650
2015-08-25,6220,92200,47166,13071,327500,253186,98418,167000,4310,372000,82500,614,4360,51700,15200,9620,60217,56295,21250,12434,3430,3895,66800,8400,29050,22750,4309,59800,41266,8530,45241,3043,2715,6550,25516,34850,315182,2940,7390,5498,...,31900,24000,166000,16400,55900,198000,23613,141000,9470,3375,41348,99300,51400,147500,10050,25024,3405,27000,121000,3015,2960,115000,19500,6180,2410,5310,2135,2790,64000,21850,4318,73678,16100,3695,20600,9470,8000,16050,20350,3985
2015-08-26,7090,91700,50001,13311,339500,270631,105675,170500,4330,390000,82500,637,4460,52900,15950,10000,60669,59367,22150,12661,3480,4195,67000,8875,29150,23600,4444,62300,42381,8790,55422,3092,3130,6350,26791,34700,326595,3005,7540,5740,...,32300,25500,168500,16950,58400,203500,24816,146000,9980,3473,44283,103500,52200,150000,10300,25771,3480,28150,121000,3052,3065,120000,19150,6300,2413,5480,2315,2835,64200,23850,4463,79004,17750,3860,20300,9300,8180,16450,21550,3905
2015-08-27,7570,91000,49743,13215,330000,267331,103861,167000,4300,393000,81000,644,4360,52700,15750,10050,60940,58373,21450,12888,3550,4130,66900,9050,30250,23550,4444,64200,42158,9140,52974,3096,3100,6390,26537,34700,323084,3050,7570,5809,...,32250,27500,167500,16750,59400,200500,25367,148000,10100,3473,43749,103000,51800,147000,10150,25332,3660,28600,126000,2983,3070,125000,19450,6520,2541,5740,2400,2905,64200,22800,4585,77229,18100,3855,21000,10850,8260,17000,21350,3925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,5850,27650,6160,7350,390500,98400,29400,181500,5040,442500,67800,1225,6720,53500,60600,8810,195000,77300,35450,7590,6680,6700,55400,35950,16350,42050,2815,36350,12900,5910,44650,9040,1825,7180,30150,50400,312500,8460,5350,7510,...,57100,10050,199000,23100,19450,276500,79700,81800,20350,6860,49800,91400,51600,226000,12900,19950,4325,24700,86200,5190,6800,92700,13400,7380,3150,10050,2795,2755,37350,22150,9120,104500,21850,11450,53000,8040,10650,13800,22400,4340
2021-07-12,5910,28550,6260,7520,395000,101000,29900,181000,4965,443000,68500,1255,6980,55000,64300,9030,195500,79200,35850,7610,6870,6630,56000,36050,16450,43600,2915,36450,13100,6000,43800,9280,1860,7300,30300,51500,311500,9150,5440,7910,...,57700,10150,200500,23150,19750,281000,80000,8

In [35]:
df2.isnull().sum()

AJ네트웍스     0
AK홀딩스      0
BGF        0
BNK금융지주    0
BYC        0
          ..
휴니드        0
휴비스        0
휴스틸        0
휴켐스        0
흥국화재       0
Length: 672, dtype: int64

In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1451 entries, 2015-08-21 to 2021-07-15
Columns: 672 entries, AJ네트웍스 to 흥국화재
dtypes: int64(672)
memory usage: 7.5 MB


In [37]:
# 결측값을 가진 열을 제거한 DataFrame을 csv 파일로 저장하기
df2.to_csv("kospi_new_name_close_drop_columns.csv", index=True)

In [38]:
# Get the assets / tickers
assets = df2.columns
assets

Index(['AJ네트웍스', 'AK홀딩스', 'BGF', 'BNK금융지주', 'BYC', 'CJ', 'CJ CGV', 'CJ대한통운',
       'CJ씨푸드', 'CJ제일제당',
       ...
       '황금에스티', '효성', '효성ITX', '후성', '휠라홀딩스', '휴니드', '휴비스', '휴스틸', '휴켐스',
       '흥국화재'],
      dtype='object', length=672)

In [39]:
len(assets)

672

In [40]:
df2.isnull().sum()

AJ네트웍스     0
AK홀딩스      0
BGF        0
BNK금융지주    0
BYC        0
          ..
휴니드        0
휴비스        0
휴스틸        0
휴켐스        0
흥국화재       0
Length: 672, dtype: int64

- Index : 날짜 (1,451  days)
- column : 종목명 (672개 종목)
- row : 날짜별 종목의 종가(Close Price)

In [41]:
!pip install PyPortfolioOpt

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 1.3MB 17.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.13-cp37-cp37m-linux_x86_64.whl size=2736209 sha256=a305c2d0e5d7d8894a2fc565f0c59cbadba65034889e23d8ce696ea9bf95dc75
  Stored in directory: /root/.cache/pip/wheels/f9/78/00/f29636789ee83434953b5442f16ec3f9834a68e7fd0393c220
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [42]:
# Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [43]:
# Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(df2)
S = risk_models.sample_cov(df2)

In [44]:
mu

AJ네트웍스    -0.023282
AK홀딩스     -0.178806
BGF       -0.296297
BNK금융지주   -0.101911
BYC        0.030191
             ...   
휴니드       -0.044305
휴비스        0.056159
휴스틸       -0.025129
휴켐스        0.023585
흥국화재       0.026178
Length: 672, dtype: float64

In [45]:
S

,AJ네트웍스,AK홀딩스,BGF,BNK금융지주,BYC,CJ,CJ CGV,CJ대한통운,CJ씨푸드,CJ제일제당,CS홀딩스,DB,DB금융투자,DB손해보험,DB하이텍,DGB금융지주,DI동일,DL,DL건설,DRB동일,DSR,DSR제강,E1,F&F홀딩스,GKL,GS건설,GS글로벌,GS리테일,HDC,HDC현대EP,HMM,HSD엔진,IHQ,JB금융지주,JW중외제약,KB금융,KCC,KCTC,KC그린홀딩스,KC코트렐,...,현대건설,현대그린푸드,현대글로비스,현대로템,현대리바트,현대모비스,현대미포조선,현대백화점,현대비앤지스틸,현대약품,현대엘리베이,현대위아,현대제철,현대차,현대차증권,현대코퍼레이션,현대퓨처넷,현대해상,현대홈쇼핑,형지엘리트,혜인,호텔신라,화성산업,화승인더,화승코퍼레이션,화신,화인베스틸,화천기계,화천기공,환인제약,황금에스티,효성,효성ITX,후성,휠라홀딩스,휴니드,휴비스,휴스틸,휴켐스,흥국화재
AJ네트웍스,0.155143,0.044273,0.042450,0.019653,0.019138,0.027426,0.039751,0.013884,0.027754,0.027773,0.021280,0.026573,0.029261,0.013748,0.038426,0.026271,0.036275,0.024508,0.018123,0.029073,0.032207,0.039969,0.025197,0.035678,0.030282,0.032341,0.040646,0.022704,0.030784,0.035078,0.028926,0.035163,0.039792,0.021801,0.034952,0.021511,0.031086,0.032669,0.036200,0.041387,...,0.036192,0.032605,0.019832,0.032636,0.027663,0.017547,0.038623,0.025812,0.029553,0.037935,0.037022,0.031672,0.039838,0.020384,0.029738,0.047218,0.023960,0.014093,0.026165,0.041946,0.023513,0.033957,0.028084,0.042092,0.026593,0.041475,0.036605,0.028757,0.019355,0.030105,0.039701,0.027140,0.025466,0.032468,0.043625,0.026238,0.030114,0.031413,0.023941,0.031113
AK홀딩스,0.044273,0.179474,0.049956,0.034863,0.018019,0.046495,0.069564,0.024779,0.032867,0.046370,0.023838,0.040326,0.051927,0.038325,0.047635,0.039316,0.044771,0.052694,0.039931,0.039890,0.057459,0.064621,0.028361,0.051466,0.058848,0.056621,0.058737,0.039472,0.054808,0.048831,0.051950,0.054119,0.057694,0.038971,0.046790,0.042987,0.051336,0.042164,0.049683,0.054060,...,0.060851,0.046095,0.035676,0.060244,0.053945,0.035963,0.057676,0.040030,0.050479,0.040255,0.056001,0.061574,0.050966,0.037134,0.048837,0.067357,0.034867,0.027185,0.036840,0.050071,0.037095,0.059848,0.043373,0.059090,0.042868,0.050182,0.061238,0.034146,0.026416,0.036320,0.058462,0.045946,0.037131,0.058688,0.050804,0.036644,0.044285,0.035875,0.041755,0.050577
BGF,0.042450,0.049956,0.192153,0.027324,0.020161,0.040520,0.047413,0.022518,0.032743,0.038587,0.019162,0.033388,0.033316,0.018984,0.038787,0.026323,0.028410,0.029496,0.031959,0.033417,0.043566,0.049507,0.022695,0.032510,0.034028,0.037655,0.044952,0.065539,0.039307,0.031256,0.035742,0.044643,0.036417,0.023417,0.056320,0.028842,0.043354,0.029118,0.037002,0.043573,...,0.035641,0.042745,0.024923,0.038646,0.044153,0.022756,0.042438,0.031029,0.031238,0.034482,0.040447,0.041575,0.034191,0.022781,0.031636,0.048652,0.022024,0.020927,0.029096,0.048939,0.017871,0.034910,0.030841,0.039707,0.030907,0.042162,0.040642,0.030105,0.013596,0.031696,0.041471,0.030772,0.031700,0.036400,0.035985,0.023135,0.030895,0.030397,0.032319,0.035209
BNK금융지주,0.019653,0.034863,0.027324,0.082042,0.015374,0.028288,0.030739,0.012217,0.009797,0.020103,0.011194,0.016597,0.038546,0.036342,0.021846,0.051069,0.015962,0.036563,0.022852,0.019723,0.022865,0.022166,0.020365,0.021612,0.028358,0.040100,0.031693,0.022389,0.035408,0.021303,0.025335,0.034023,0.022654,0.037483,0.017177,0.050575,0.036929,0.017815,0.024332,0.021366,...,0.040188,0.026874,0.025228,0.024577,0.028511,0.027004,0.045636,0.028199,0.026121,0.014163,0.022041,0.033132,0.044953,0.027595,0.039053,0.037553,0.010625,0.032949,0.021179,0.016600,0.016453,0.024717,0.024479,0.027547,0.025544,0.025939,0.031734,0.022723,0.017990,0.013317,0.023603,0.025954,0.015686,0.022558,0.029629,0.020163,0.020639,0.024812,0.028361,0.028252
BYC,0.019138,0.018019,0.020161,0.015374,0.066651,0.018956,0.023139,0.006667,0.014575,0.013888,0.015259,0.008562,0.019484,0.013199,0.017289,0.015728,0.016436,0.020967,0.016147,0.017379,0.020469,0.024507,0.014551,0.014894,0.019045,0.017759,0.020523,0.011928,0.016835,0.016171,0.025089,0.017191,0.021092,0.013214,0.018792,0.015987,0.019495,0.016715,0.022045,0.017714,...,0.022138,0.017242,0.011466,0.026481,0.022441,0.015555,0.021876,0.014697,0.014259,0.015848,0.028815,0.020635,0.020757,0.015285,0.018743,0.027279,0.011022,0.011635,0.013152,0.023421,0.015374,0.017250,0.017450,0.023598,0.022999,0.022374,0.019692,0.019206,0.012896,0.0161

In [46]:
S.shape

(672, 672)

In [47]:
# Optimize for the maximal Sharpe ratio
# 💛데이터셋이 너무 많으면, ef.max_sharpe()에서 에러남 -> solver를 SCS로 바꿔줌
# Rober says: 100개 이하로 종목을 추린 후에 실행시키기를 추천함 !
ef = EfficientFrontier(mu, S, solver="SCS")  # Create the Efficient Frontier Object

# Maximize the Sharpe ratio, and get the raw weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

ef.portfolio_performance(verbose=True)

OrderedDict([('AJ네트웍스', 0.0), ('AK홀딩스', 0.0), ('BGF', 0.0), ('BNK금융지주', 0.0), ('BYC', 0.0), ('CJ', 0.0), ('CJ CGV', 0.0), ('CJ대한통운', 0.0), ('CJ씨푸드', 0.0), ('CJ제일제당', 0.0), ('CS홀딩스', 0.0), ('DB', 0.0), ('DB금융투자', 0.0), ('DB손해보험', 0.0), ('DB하이텍', 0.0), ('DGB금융지주', 0.0), ('DI동일', 0.0), ('DL', 0.0), ('DL건설', 0.0), ('DRB동일', 0.0), ('DSR', 0.0), ('DSR제강', 0.0), ('E1', 0.0), ('F&F홀딩스', 0.0), ('GKL', 0.0), ('GS건설', 0.0), ('GS글로벌', 0.0), ('GS리테일', 0.0), ('HDC', 0.0), ('HDC현대EP', 0.0), ('HMM', 0.0), ('HSD엔진', 0.0), ('IHQ', 0.0), ('JB금융지주', 0.0), ('JW중외제약', 0.0), ('KB금융', 0.0), ('KCC', 0.0), ('KCTC', 0.0), ('KC그린홀딩스', 0.0), ('KC코트렐', 0.0), ('KEC', 0.0), ('KG동부제철', 0.0), ('KG케미칼', 0.0), ('KH 필룩스', 0.0), ('KISCO홀딩스', 0.0), ('KPX케미칼', 0.0), ('KPX홀딩스', 0.0), ('KR모터스', 0.0), ('KT', 0.0), ('KT&G', 0.0), ('KTB투자증권', 0.0), ('KTcs', 0.0), ('KTis', 0.0), ('LF', 0.0), ('LG', 0.0), ('LG디스플레이', 0.0), ('LG생활건강', 0.0), ('LG유플러스', 0.0), ('LG이노텍', 0.0), ('LG전자', 0.0), ('LG헬로비전', 0.0), ('LG화학', 0.0), ('LS', 0.0), 

(0.3533455898283326, 0.13780941817475195, 2.418888304176911)

In [48]:
weights

OrderedDict([('AJ네트웍스', 4.9666625565e-06),
             ('AK홀딩스', -6.372239596e-07),
             ('BGF', -2.0230604146e-06),
             ('BNK금융지주', -2.9696602764e-06),
             ('BYC', -3.5644212653e-06),
             ('CJ', -1.3450736706e-06),
             ('CJ CGV', 2.0398625072e-06),
             ('CJ대한통운', -1.4409151772e-06),
             ('CJ씨푸드', 3.3684094204e-06),
             ('CJ제일제당', -2.7599042207e-06),
             ('CS홀딩스', 1.8220630511e-06),
             ('DB', -3.197684827e-07),
             ('DB금융투자', 9.442207913e-07),
             ('DB손해보험', -2.6061346224e-06),
             ('DB하이텍', 2.1594473368e-06),
             ('DGB금융지주', -9.783204711e-07),
             ('DI동일', 7.10141686e-08),
             ('DL', -1.2490367143e-06),
             ('DL건설', 6.391078926e-07),
             ('DRB동일', 1.7090481135e-06),
             ('DSR', -7.090725763e-07),
             ('DSR제강', -1.808601053e-07),
             ('E1', 7.807591033e-07),
             ('F&F홀딩스', -1.5277193179e-06

In [49]:
cleaned_weights

OrderedDict([('AJ네트웍스', 0.0),
             ('AK홀딩스', 0.0),
             ('BGF', 0.0),
             ('BNK금융지주', 0.0),
             ('BYC', 0.0),
             ('CJ', 0.0),
             ('CJ CGV', 0.0),
             ('CJ대한통운', 0.0),
             ('CJ씨푸드', 0.0),
             ('CJ제일제당', 0.0),
             ('CS홀딩스', 0.0),
             ('DB', 0.0),
             ('DB금융투자', 0.0),
             ('DB손해보험', 0.0),
             ('DB하이텍', 0.0),
             ('DGB금융지주', 0.0),
             ('DI동일', 0.0),
             ('DL', 0.0),
             ('DL건설', 0.0),
             ('DRB동일', 0.0),
             ('DSR', 0.0),
             ('DSR제강', 0.0),
             ('E1', 0.0),
             ('F&F홀딩스', 0.0),
             ('GKL', 0.0),
             ('GS건설', 0.0),
             ('GS글로벌', 0.0),
             ('GS리테일', 0.0),
             ('HDC', 0.0),
             ('HDC현대EP', 0.0),
             ('HMM', 0.0),
             ('HSD엔진', 0.0),
             ('IHQ', 0.0),
             ('JB금융지주', 0.0),
             ('JW중외제약', 0.0),

In [50]:
!pip install pulp

     |████████████████████████████████| 40.6MB 78kB/s 


In [51]:
# Get the discrete allocation of each sharpe per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

# 투자금액 (단위: KRW)
portfolio_val = 15000000

latest_prices = get_latest_prices(df2)

weights = cleaned_weights

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_val)

allocation, leftover = da.lp_portfolio()

print('Discrete Allocaion: ', allocation)
print('Funds Remaining: ', leftover, ' KRW')

Discrete Allocaion:  {'NAVER': 1, '그린케미칼': 12, '넥스트사이언스': 12, '대성홀딩스': 63, '대한방직': 6, '동남합성': 24, '동방': 7, '맥쿼리인프라': 134, '미원상사': 15, '부산주공': 2, '삼성SDI': 1, '삼성공조': 8, '삼성전자': 3, '삼화콘덴서': 4, '시디즈': 5, '신풍제약': 7, '일진머티리얼즈': 1, '카카오': 5, '코스모신소재': 8, '포스코강판': 7, '포스코케미칼': 5, '한미반도체': 2, '한전산업': 11}
Funds Remaining:  28.0  KRW


In [52]:
allocation

{'NAVER': 1,
 '그린케미칼': 12,
 '넥스트사이언스': 12,
 '대성홀딩스': 63,
 '대한방직': 6,
 '동남합성': 24,
 '동방': 7,
 '맥쿼리인프라': 134,
 '미원상사': 15,
 '부산주공': 2,
 '삼성SDI': 1,
 '삼성공조': 8,
 '삼성전자': 3,
 '삼화콘덴서': 4,
 '시디즈': 5,
 '신풍제약': 7,
 '일진머티리얼즈': 1,
 '카카오': 5,
 '코스모신소재': 8,
 '포스코강판': 7,
 '포스코케미칼': 5,
 '한미반도체': 2,
 '한전산업': 11}

In [53]:
len(allocation)

23

In [54]:
# 포트폴리오에 포함된 종목을 리스트로 만들기
company_name = list(allocation)

In [55]:
company_name

['NAVER',
 '그린케미칼',
 '넥스트사이언스',
 '대성홀딩스',
 '대한방직',
 '동남합성',
 '동방',
 '맥쿼리인프라',
 '미원상사',
 '부산주공',
 '삼성SDI',
 '삼성공조',
 '삼성전자',
 '삼화콘덴서',
 '시디즈',
 '신풍제약',
 '일진머티리얼즈',
 '카카오',
 '코스모신소재',
 '포스코강판',
 '포스코케미칼',
 '한미반도체',
 '한전산업']

In [56]:
len(company_name)

23

In [57]:
# Get the discrete allocation values
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [58]:
discrete_allocation_list

[1, 12, 12, 63, 6, 24, 7, 134, 15, 2, 1, 8, 3, 4, 5, 7, 1, 5, 8, 7, 5, 2, 11]

In [59]:
# Create a dataframe for the portfolio
portfolio_df = pd.DataFrame(columns = ['Company_name', 'company_Ticker', 'Discrete_val_'+str(portfolio_val)])
portfolio_df

,Company_name,company_Ticker,Discrete_val_15000000


In [60]:
portfolio_df['Company_name'] = company_name
portfolio_df['company_Ticker'] = allocation
portfolio_df['Discrete_val_'+str(portfolio_val)] = discrete_allocation_list

In [61]:
portfolio_df

,Company_name,company_Ticker,Discrete_val_15000000
0,NAVER,NAVER,1
1,그린케미칼,그린케미칼,12
2,넥스트사이언스,넥스트사이언스,12
3,대성홀딩스,대성홀딩스,63
4,대한방직,대한방직,6
5,동남합성,동남합성,24
6,동방,동방,7
7,맥쿼리인프라,맥쿼리인프라,134
8,미원상사,미원상사,15
9,부산주공,부산주공,2


In [62]:
# Sort by allocation & Show the portfolio
portfolio_df_sorted = portfolio_df.sort_values('Discrete_val_'+str(portfolio_val), ascending = False)
portfolio_df_sorted = portfolio_df_sorted.reset_index(drop=True)
portfolio_df_sorted

,Company_name,company_Ticker,Discrete_val_15000000
0,맥쿼리인프라,맥쿼리인프라,134
1,대성홀딩스,대성홀딩스,63
2,동남합성,동남합성,24
3,미원상사,미원상사,15
4,넥스트사이언스,넥스트사이언스,12
5,그린케미칼,그린케미칼,12
6,한전산업,한전산업,11
7,코스모신소재,코스모신소재,8
8,삼성공조,삼성공조,8
9,동방,동방,7


In [63]:
# Show Funds Remaining
print('Funds Remaining: ', leftover, ' KRW')

# Show Portfolio performance
ef.portfolio_performance(verbose=True)

Funds Remaining:  28.0  KRW
Expected annual return: 35.3%
Annual volatility: 13.8%
Sharpe Ratio: 2.42


(0.3533455898283326, 0.13780941817475195, 2.418888304176911)